In [108]:
from collections     import defaultdict, Counter
from collections.abc import Mapping
from itertools       import product
from operator        import add
from typing          import Generator, List
from helpers         import data

In [75]:
start = data(17)
start

['#.#####.',
 '#..##...',
 '.##..#..',
 '#.##.###',
 '.#.#.#..',
 '#.##..#.',
 '#####..#',
 '..#.#.##']

In [76]:
len(start), len(start[0])

(8, 8)

A **neighbor** is any cube where **any coordinate differs by at most 1**. 

Active cubes are denoted with \#, and inactive cubes are denoted with \. 

The boot process consists of six cycles. During each cycle: 
- If a cube is **active** and exactly **2 or 3 neighbors are active**, it stays active. Otherwise it becomes inactive. 
- If a cube is **inactive** and exactly **3 neighors are active**, it becomes active. Otherwise it stays inactive. 

**Part 1:** How many cubes are left in the active state after the sixth cycle?

**IGNORE THIS!** Scroll down to see an actual working solution. 

In [77]:
def nested_get(d, *keys):
    if isinstance(d, Mapping):
        first, *rest = keys
        return nested_get(d[first], *rest)
    return d

In [78]:
class MutableBool:
    def __init__(self, val=False):
        self.val = val 
        self.next = False
        
    def __bool__(self):
        return self.val
    
    def __repr__(self):
        return str(self.val)
    
    def update(self):
        self.val, self.next = self.next, False

In [79]:
# At each time step, update each entry based on the rule and its neighbors 
class NGrid(dict):
    """NGrid will choose default values for missing keys and is nested n deep."""

    # Tracks minimum and maximum range for each level 
    ranges: dict[int, List[int]] = {}

    def __init__(self, n):
        self.n = n
        self.ranges[self.n] = [0, 0] # Will always be overridden 
        
    def __missing__(self, key): 
        if self.n == 1:
            val = MutableBool()
        else: 
            val = NGrid(self.n - 1)
        self[key] = val
            
        # Update minimum or maximum 
        if self.ranges[self.n][0] > key:
            self.ranges[self.n][0] = key
        if self.ranges[self.n][1] <= key: 
            self.ranges[self.n][1] = key + 1
        
        return val
            
    def life(self, steps): 
        """Simulate steps iterations of Conway's Game of Life."""
        for i in range(steps): 
            self.step()
    
    def step(self):
        """Simulate one iteration of Conway's Game of Life."""
        periods = list(map(lambda r: (r[0] - 1, r[1] + 1), self.ranges.values()))
        for coords in product(*[range(*period) for period in periods]):
            neighbors_on = sum(map(bool, self.neighbors(*coords)))
            if neighbors_on == 3: 
                nested_get(self, *coords).next = True 
            elif neighbors_on == 2 and not nested_get(self, *coords):
                nested_get(self, *coords).next = True 
            else: 
                nested_get(self, *coords).next = False 
        
        for coords in product(*[range(*period) for period in periods]):
            nested_get(self, *coords).update()
    
    def neighbors(self, *args, center=True) -> Generator[bool, None, None]:
        """Yield neighbors of coordinate at self[args[0]][args[1]][...]
        
        `center` tracks whether we've offset at least once during recursion. We 
        need to know this so we can ensure we never print out the element whose 
        neighbors we're searching for. 
        """
        if len(args) != self.n: 
            raise ValueError(f"Expected {self.n} integer coordinates, got {len(args)}.")
        if any(type(arg) != int for arg in args):
            raise ValueError("Expected integer coordinates.")
            
        first, *rest = args
        if self.n == 1:
            yield self[first - 1]
            yield self[first + 1]
            if not center:
                yield self[first]
        else: 
            yield from self[first - 1].neighbors(*rest, center=False)
            yield from self[first].neighbors(*rest, center=center)
            yield from self[first + 1].neighbors(*rest, center=False)

In [80]:
g = NGrid(3)

In [67]:
for y, l in enumerate(start): 
    for x, c in enumerate(l):
        if c == "#":
            g[0][y][x] = MutableBool(True)

In [68]:
g.life(6)

In [39]:
def count(d): 
    if isinstance(d, Mapping):
        return sum(count(v) for k, v in d.items())
    else:
        if bool(d):
            return 1
        return 0

In [40]:
count(g)

16

My code is getting too complex and is wrong. I peeked at Norvig's solution and I see he's using `set`s, so I'll try starting with that and see where it goes. 

In [106]:
Cube = tuple

def parse_image(image: List[str], active="#", d=3) -> set[Cube]:
    """Parse image as a starting point for d-dimensional Conway game."""
    return {(x, y, *(0,) * (d-2)) 
             for y in range(len(image))
             for x, c in enumerate(image[y]) 
             if c == active}

In [115]:
def simulate(cubes: set[Cube]) -> set[Cube]:
    """Simulate a time step according to set rules.
    TODO: Rules should be changeable!
    """
    n_neighbors = count_neighbors(cubes)
    return {neighbor 
            for neighbor, n in n_neighbors.items()
            if n == 3 
            or n == 2 and neighbor in cubes}


def offsets(d: int) -> Generator[tuple, None, None]:
    """Generate offsets to apply to each cell."""
    # I had to peek at Norvig's code again because I remembered he did something 
    # similar, and I was having a bug. The problem was I forgot to exclude the 
    # (0, 0, 0) offset via any. 
    yield from filter(any, product((-1, 0, 1), repeat=d))

def count_neighbors(cubes: set[Cube]) -> dict[Cube, int]:
    """Generate mapping from Cube to number of neighbors."""       
    n_neighbors = defaultdict(int)
    for cube in cubes: 
        # Add each active cube's neighbors 
        for offset in offsets(len(cube)):
            neighbor = tuple( map(add, cube, offset) ) # Add elements of tuple 
            n_neighbors[neighbor] += 1
    return n_neighbors

In [116]:
def life(cubes: set[Cube], steps: int) -> set[Cube]:
    """Simulate for `steps` iterations."""
    for step in range(steps): 
        cubes = simulate(cubes)
    return cubes

def count_active(cubes: set[Cube]) -> int: 
    return len(cubes)

In [117]:
# Active cubes
cubes = parse_image(start)
assert (0, 0, 0) in cubes

count_active(life(cubes, steps=6))

353

**Part 2:** Simulate 6 steps in a 4D space. 

In [118]:
cubes = parse_image(start, d=4)

count_active(life(cubes, steps=6))

2472

**Norvig:** The biggest thing I'm taking away is that there's always a simpler way to set up a problem... I'm also surprised at how well he knows `itertools`. Another small trick is to use `Counter` instead of a `defaultdict` that I increment.

In [113]:
def count_neighbors(cubes: set[Cube]) -> dict[Cube, int]:
    """Generate mapping from Cube to number of neighbors."""       
    return Counter(tuple( map(add, cube, offset) )
                   for cube in cubes
                   for offset in offsets(len(cube)))